In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


In [17]:
import time
import json
import pickle
import numpy as np
import random
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow
import logging
logging.getLogger('tensorflow').disabled = True

In [18]:
from scraper import Scraper
# import pyttsx3

# engine = pyttsx3.init() 
def print_and_speak(text):
    print(text)
    # engine.say(text)
    # engine.runAndWait()

In [19]:
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [20]:
# clean sentence to words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    '''print(sentence_words)'''
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    '''print(sentence_words)'''
    return sentence_words

In [21]:
# return bag of words array: 0 or 1 for each word in the bag that exist in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    '''print("bag: ",bag)'''
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

In [22]:
def predict_class(sentence, model):
    # filter out predictions below a threashold
    p = bow(sentence, words, show_details=False)
    '''print(p)'''
    res = model.predict(np.array([p]))[0]
    '''print('res: ',res)'''
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    '''print(results)'''
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    '''print(return_list)'''
    return return_list

In [23]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    '''print('tag -> ',tag)'''
    list_of_intents = intents_json['intents']
    '''print('intent_json-> ',intents_json)'''
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

In [24]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return (ints,res)

In [25]:
class finalRes:
    
    def __init__(self,input_,ints,res):
        self.tag = ints[0]['intent']
        self.res = res
        
        if self.tag == 'search':
            self.tagSearch()
        elif self.tag == 'test':
            self.tagTest()
    
    def tagSearch(self):
        print_and_speak(self.res+' ')
        queryTerm = input()
        o = Scraper(queryTerm)
        try:
            print_and_speak(o.definition())
            print('='*100,'Here are some resources','='*100,sep='\n')
            for key, value in o.resources().items():
                print(key+' : '+ ' , '.join(value))
            print('='*100)
        except Exception as e:
            print(e,'='*100,end='\n')
            print_and_speak("Sorry, I can't get you try asking something else.")
            main()
            
    def tagTest(self):
        marks = 0
        print_and_speak(self.res+' ')
        queryTerm = input()
        o = Scraper(queryTerm)
        try:
            mcqs =  o.mcqs()
            for q, op in mcqs.items():
                print(q, '\noptions :-', '\n'.join(mcqs[q]['options']), sep='\n', end='\n')
                print('\nAnswer - ', end = ' ')
                res = input().lower()
                if res == mcqs[q]['answer']:
                    marks+=1
                    print_and_speak('[✓] Right answer\n')
                    print('='*30, mcqs[q]['explain'], '='*30, sep='\n')
                    print()
                    time.sleep(2)
                else:
                    print_and_speak('[✕] Wrong answer\n')
                    print('Answer: ', mcqs[q]['answer'],'\n')
                    print('='*30, mcqs[q]['explain'], '='*30, sep='\n')
                    print()
                    time.sleep(2)
            print_and_speak("Score: "+ str(marks)+ '/'+ str(len(mcqs)))
        except Exception as e:
            print(e,'='*100,end='\n')
            print_and_speak("Sorry, I can't get you try asking something else.")
            main()            

In [28]:
def main():
    input_ = input()
    if input_.lower() == 'exit':
        print_and_speak('Glad to see you again. Bye!')
        sys.exit(0)    
    elif not input_:
        print_and_speak("Don't nullify! I'm here to help you")
        main()
    else:
        ints, res = chatbot_response(input_)
        #print(ints, res)
        tag = ints[0]['intent']
        if tag not in ['greeting','goodbye','thanks','options']:
            finalRes(input_,ints, res)
        else:
            print_and_speak(res)
            if tag == 'goodbye':
                sys.exit(0)

In [29]:
if __name__ == '__main__':
    while True:
        try:
            main()
        except KeyboardInterrupt:
            print_and_speak('Exiting.. Have a nice day!')
            sys.exit(0)

1/1 [==============================] - 0s 88ms/step
Hello, thanks for asking
1/1 [==============================] - 0s 33ms/step
Topic :  
1. Which of the following is the most widely used external memory data structure?

options :-
a) AVL tree
b) B-tree
c) Red-black tree
d) Both AVL tree and Red-black tree

Answer -  [✕] Wrong answer

Answer:  b 

Explanation: In external memory, the data is transferred in form of blocks. These blocks have data valued and pointers. And B-tree can hold both the data values and pointers. So B-tree is used as an external memory data structure.

2. B-tree of order n is a order-n multiway tree in which each non-root node contains __________

options :-
a) at most (n – 1)/2 keys
b) exact (n – 1)/2 keys
c) at least 2n keys
d) at least (n – 1)/2 keys

Answer -  [✕] Wrong answer

Answer:  d 

Explanation: A non-root node in a B-tree of order n contains at least (n – 1)/2 keys. And contains a maximum of (n – 1) keys and n sons.

3. A B-tree of order 4 and of he

SystemExit: 0

/home/kali/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# chatbot that helps in your studies